In [1]:
!unzip "/content/drive/MyDrive/Dataset/indoor_CVPR_09_augmented.zip"

Streaming output truncated to the last 5000 lines.
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_0_museo_158_20_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_1_museo_158_20_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_2_museo_158_20_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_3_museo_158_20_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_4_museo_158_20_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_0_museo_87_22_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented/museum/aug_1_museo_87_22_flickr.jpg  
  inflating: content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_

In [2]:
from sklearn.model_selection import train_test_split
import os
from PIL import Image
import numpy as np
import random

In [3]:
batch_size = 32
img_height = 224
img_width = 224

In [4]:
import cv2
from sklearn.model_selection import train_test_split

In [5]:
def load_dataset(dataset_path, test_size=0.2, random_state=42):
    categories = os.listdir(dataset_path)
    X = []
    y = []

    for category in categories:
        category_path = os.path.join(dataset_path, category)
        if os.path.isdir(category_path):
            images = os.listdir(category_path)
            for image in images:
                image_path = os.path.join(category_path, image)
                img = Image.open(image_path)
                img = np.array(img)
                X.append(img)
                y.append(category)

    X = np.array(X)
    y = np.array(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

In [6]:
DATADIR = "/content/content/drive/MyDrive/Datasets/MITIndoorDataset/indoor_CVPR_09_augmented"
CATEGORIES = os.listdir(DATADIR)

In [7]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = load_dataset(DATADIR)

# Print the shapes of the training and testing sets
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (62456, 224, 224, 3)
Shape of y_train: (62456,)
Shape of X_test: (15614, 224, 224, 3)
Shape of y_test: (15614,)


In [8]:
# from keras.utils import to_categorical

# # Assuming your target labels are currently in a list called target_labels
# target_labels = CATEGORIES  # Fill this with your actual target labels

# # Convert target labels to one-hot encoded format
# one_hot_target = to_categorical(target_labels, num_classes=67)

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Assuming your target labels are currently stored in a list called target_labels
target_labels = CATEGORIES  # Fill this with your actual target labels

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform target labels
encoded_labels_train = label_encoder.fit_transform(y_train)
encoded_labels_test = label_encoder.fit_transform(y_test)

# Convert encoded labels to one-hot encoded format
y_train = to_categorical(encoded_labels_train, num_classes=len(label_encoder.classes_))
y_test = to_categorical(encoded_labels_test, num_classes=len(label_encoder.classes_))

In [9]:

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (62456, 224, 224, 3)
Shape of y_train: (62456, 67)
Shape of X_test: (15614, 224, 224, 3)
Shape of y_test: (15614, 67)


In [9]:
import numpy as np

import numpy as np

def top_k_accuracy(predictions, labels, k=1):
    """
    Calculate the top-k accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).
    k: Integer, denotes top-k accuracy.

    Returns:
    accuracy: Float, top-k accuracy.
    """
    sorted_indices = np.argsort(predictions, axis=1)[:, ::-1]
    top_k_indices = sorted_indices[:, :k]
    top_k_labels = np.take_along_axis(labels, top_k_indices, axis=1)
    correct_predictions = np.sum(top_k_labels, axis=1)
    accuracy = np.mean(correct_predictions)
    return accuracy


def top_1_accuracy(predictions, labels):
    """
    Calculate the top-1 accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    accuracy: Float, top-1 accuracy.
    """
    return top_k_accuracy(predictions, labels, k=1)

def top_5_accuracy(predictions, labels):
    """
    Calculate the top-5 accuracy.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    accuracy: Float, top-5 accuracy.
    """
    return top_k_accuracy(predictions, labels, k=5)

def top_k_error(predictions, labels, k=1):
    """
    Calculate the top-k error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).
    k: Integer, denotes top-k error.

    Returns:
    error: Float, top-k error rate.
    """
    return 1.0 - top_k_accuracy(predictions, labels, k)

def top_1_error(predictions, labels):
    """
    Calculate the top-1 error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    error: Float, top-1 error rate.
    """
    return top_k_error(predictions, labels, k=1)

def top_5_error(predictions, labels):
    """
    Calculate the top-5 error rate.

    Args:
    predictions: Array-like, predicted binary values (0 or 1).
    labels: Array-like, ground truth binary values (0 or 1).

    Returns:
    error: Float, top-5 error rate.
    """
    return top_k_error(predictions, labels, k=5)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [11]:
from keras.models import load_model

stage2 = load_model("/content/drive/MyDrive/Trained Models/MIT/resnet18_with_cbam_after_stage_2_MIT_val_accuracy.h5")
y_pred = stage2.predict(X_test)

threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

488/488 [==============================] - 43s 77ms/step
Accuracy: 0.8925323427693096
Top-1 Accuracy: 0.89253235
Top-5 Accuracy: 0.8968874
Top-1 Error: 0.1074676513671875
Top-5 Error: 0.10311257839202881
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.89       606
           1       0.87      0.79      0.83       141
           2       0.92      0.88      0.90       173
           3       0.83      0.85      0.84       376
           4       0.87      0.88      0.87       592
           5       0.92      0.87      0.90       195
           6       0.89      0.92      0.90       632
           7       0.89      0.94      0.91       375
           8       0.94      0.96      0.95       210
           9       0.92      0.88      0.90       119
          10       0.94      0.94      0.94       481
          11       0.91      0.87      0.89        97
          12       0.94      0.94      0.94       212
          13    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
stage3 = load_model("/content/drive/MyDrive/Trained Models/MIT/resnet18_with_cbam_after_stage_3_MIT_val_accuracy.h5")
y_pred = stage3.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

488/488 [==============================] - 37s 75ms/step
Accuracy: 0.887664916100935
Top-1 Accuracy: 0.8879211
Top-5 Accuracy: 0.89387727
Top-1 Error: 0.11207890510559082
Top-5 Error: 0.10612273216247559
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       606
           1       0.90      0.72      0.80       141
           2       0.94      0.92      0.93       173
           3       0.88      0.90      0.89       376
           4       0.85      0.88      0.86       592
           5       0.92      0.94      0.93       195
           6       0.90      0.94      0.92       632
           7       0.84      0.86      0.85       375
           8       0.96      0.93      0.94       210
           9       0.94      0.87      0.90       119
          10       0.92      0.94      0.93       481
          11       0.92      0.86      0.89        97
          12       0.95      0.92      0.93       212
          13    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
stage4 = load_model("/content/drive/MyDrive/Trained Models/MIT/resnet18_with_cbam_after_stage_4_MIT_val_accuracy.h5")
y_pred = stage4.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

488/488 [==============================] - 38s 76ms/step
Accuracy: 0.8883694120660945
Top-1 Accuracy: 0.88836944
Top-5 Accuracy: 0.8946458
Top-1 Error: 0.11163055896759033
Top-5 Error: 0.1053541898727417
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.90      0.89       606
           1       0.87      0.74      0.80       141
           2       0.91      0.90      0.90       173
           3       0.86      0.88      0.87       376
           4       0.86      0.90      0.88       592
           5       0.96      0.92      0.94       195
           6       0.90      0.91      0.90       632
           7       0.88      0.86      0.87       375
           8       0.95      0.94      0.95       210
           9       0.94      0.87      0.90       119
          10       0.93      0.95      0.94       481
          11       0.88      0.84      0.86        97
          12       0.92      0.91      0.92       212
          13    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
stage5 = load_model("/content/drive/MyDrive/Trained Models/MIT/resnet18_with_cbam_after_stage_5_MIT_val_accuracy.h5")
y_pred = stage5.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

488/488 [==============================] - 37s 75ms/step
Accuracy: 0.8824772639938516
Top-1 Accuracy: 0.8824773
Top-5 Accuracy: 0.8886256
Top-1 Error: 0.1175227165222168
Top-5 Error: 0.1113743782043457
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.87      0.88       606
           1       0.82      0.68      0.74       141
           2       0.95      0.91      0.93       173
           3       0.83      0.88      0.85       376
           4       0.85      0.88      0.86       592
           5       0.91      0.90      0.90       195
           6       0.88      0.91      0.89       632
           7       0.86      0.85      0.85       375
           8       0.92      0.91      0.92       210
           9       0.91      0.89      0.90       119
          10       0.92      0.94      0.93       481
          11       0.85      0.89      0.87        97
          12       0.94      0.91      0.92       212
          13      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
stageEvery = load_model("/content/drive/MyDrive/Trained Models/MIT/resnet18_with_cbam_after_every_stage_MIT_val_accuracy.h5")
y_pred = stageEvery.predict(X_test)
threshold = 0.5
y_pred_binary = np.where(y_pred > threshold, 1, 0)
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)
print("Top-1 Accuracy:", top_1_accuracy(y_pred_binary, y_test))
print("Top-5 Accuracy:", top_5_accuracy(y_pred_binary, y_test))
print("Top-1 Error:", top_1_error(y_pred_binary, y_test))
print("Top-5 Error:", top_5_error(y_pred_binary, y_test))
print("Classification Report:")
print(classification_report(y_test, y_pred_binary))

488/488 [==============================] - 47s 95ms/step
Accuracy: 0.8891379531189958
Top-1 Accuracy: 0.8893301
Top-5 Accuracy: 0.89413345
Top-1 Error: 0.11066991090774536
Top-5 Error: 0.10586655139923096
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.89      0.88       606
           1       0.89      0.79      0.83       141
           2       0.94      0.88      0.91       173
           3       0.85      0.84      0.85       376
           4       0.86      0.88      0.87       592
           5       0.93      0.87      0.90       195
           6       0.87      0.89      0.88       632
           7       0.87      0.91      0.89       375
           8       0.89      0.92      0.91       210
           9       0.94      0.87      0.90       119
          10       0.94      0.95      0.95       481
          11       0.87      0.86      0.86        97
          12       0.96      0.93      0.95       212
          13   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
